# Plotting Results of Crypto Strategy


In [ ]:
# from config import general as config
# from config import crypto
import pandas as pd
import numpy as np
# matplotlib.use('Agg')
%matplotlib inline
import pyfolio
import matplotlib.pyplot as plt
from pyfolio import timeseries
from datetime import datetime
import empyrical as ep
# from finrl.plot import backtest_stats, get_daily_return # , backtest_plot, get_daily_return, get_baseline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def backtest_stats_intraday(daily_returns):
    # pyfolio perf_stats() doesnt handle 365-day data for a year
    # perf_stats_all = timeseries.perf_stats(returns=daily_returns, positions=None, transactions=None, turnover_denom="AGB")   
    stats = {}
    stats['min_date'] = str(daily_returns.index[0].strftime("%Y-%m-%d %H:%M"))
    stats['max_date'] = str(daily_returns.index[-1].strftime("%Y-%m-%d %H:%M"))
    stats['cum_ret'] = (daily_returns + 1).prod() - 1
    stats['ann_ret'] = ep.annual_return(daily_returns, annualization=365)    
    stats['ann_vol'] = ep.annual_volatility(daily_returns, annualization=365)
    stats['sharpe'] = ep.sharpe_ratio(daily_returns, risk_free=0, annualization=365)
    stats['sortino'] = ep.sortino_ratio(daily_returns, annualization=365)    
    stats['max_dd'] = ep.max_drawdown(daily_returns)    
    return stats

def print_baseline(name, stats):
    print("============== Baseline Benchmark ===========")
    print(f"Model:        {name}")
    print(f"Date Range:   {stats['min_date']} - {stats['max_date']}")
    print(f"-----------------------------------------------------")    
    print(f"Cumulative Return :   {stats['cum_ret']:7.4f}")
    print(f"Annual Return     :   {stats['ann_ret']:7.4f}")
    print(f"Annual Volatility :   {stats['ann_vol']:7.4f}")
    print(f"Sharpe ratio      :   {stats['sharpe']:7.4f}")
    print(f"Sortino ratio     :   {stats['sortino']:7.4f}")
    print(f"Max Drawdown      :   {stats['max_dd']:7.4f}")
    
def generate_backtest_results_crypto(acc_val, strat_name, model_name, run_name, printstats=True):
    
    acc_val['date'] = pd.to_datetime(acc_val['date'])
    time_res = get_time_resolution(acc_val.iloc[0]['date'], acc_val.iloc[1]['date'])
    
    print("============== Backtest Results ===========")
    daily_returns = get_daily_return_from_intraday(acc_val, value_col_name="account_value")
    stats = backtest_stats_intraday(daily_returns)
    
    parsed_stats = {}
    parsed_stats['strategy'] = strat_name
    parsed_stats['model'] = model_name
    parsed_stats['run_name'] = run_name
    parsed_stats['min_date'] = stats['min_date']
    parsed_stats['max_date'] = stats['max_date']
    parsed_stats['res'] = time_res
    parsed_stats['cum_ret'] = stats['cum_ret']
    parsed_stats['ann_ret'] = stats['ann_ret']
    parsed_stats['ann_vol'] = stats['ann_vol']
    parsed_stats['sharpe'] = stats['sharpe']
    parsed_stats['sortino'] = stats['sortino']
    parsed_stats['max_dd'] = stats['max_dd']
    parsed_stats['created'] = datetime.now().strftime("%Y-%m-%d %H:%M")

    if printstats:
        print_stats(parsed_stats)
    # append_stats(parsed_stats)
    # perf_stats_df = pd.DataFrame(perf_stats_intraday)
    # perf_stats_df.to_csv(f"{RESULTS_FILE_PREFIX}_perf_stats.csv")
    return stats, daily_returns, parsed_stats

def get_baseline(names, return_idx=0):
    single_baselines = []
    start = '2022-06-22'
    end = '2022-12-01'
    selected_baseline = None

    for i, baseline_name in enumerate(names):
        baseline_returns = get_baseline_returns_crypto(baseline_name, start, end)
        if i == return_idx:
            selected_baseline = baseline_returns
        x = pd.DataFrame(backtest_stats_intraday(baseline_returns), index=[baseline_name])
        single_baselines.append(x)

    baseline_stats = pd.concat(single_baselines)
    return baseline_stats, selected_baseline

def get_baseline_returns_crypto(baseline_strategy='MPT', start='2022-06-01 00:00:00', end='2022-11-01 23:59:59'):
    df = pd.read_csv(f"../datasets/thesis/benchmark/1d_benchmark_returns.csv", index_col=0)
    df.index = pd.to_datetime(df.index)
    baseline = df[baseline_strategy]
    baseline = baseline -1
    baseline = baseline[(baseline.index >= start ) & (baseline.index <= end)]
    return baseline

In [ ]:
RESULTS_PREFIX = '../results/benchmark/BAH'
baseline_index = 1
baseline_stats, baseline_returns = get_baseline(['MPT', 'BAH', 'CRP'], baseline_index)
# baseline_stats.to_csv(f"results/baseline.csv")
# pd.concat([pd.DataFrame(stats, index=['strategy']), baseline_stats]).T
print_baseline(baseline_returns.name, baseline_stats.iloc[baseline_index])

In [ ]:
baseline_stats.to_csv('../results/benchmark/benchmark_performance.csv')

In [ ]:
baseline_stats

In [ ]:
pyfolio.plotting.show_worst_drawdown_periods(baseline_returns)

In [ ]:
pyfolio.tears.create_interesting_times_tear_sheet(baseline_returns, set_context=False)

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,5))
pyfolio.plotting.plot_drawdown_periods(baseline_returns, top=5, ax=ax)
plt.savefig(f"{RESULTS_PREFIX}_stats_drawdown_periods.png")

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,5))
ax = pyfolio.plotting.plot_drawdown_underwater(returns=baseline_returns, ax=ax)
plt.savefig(f"{RESULTS_PREFIX}_stats_underwater.png")

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))
ax = pyfolio.plotting.plot_monthly_returns_heatmap(baseline_returns, ax=ax)